In [1]:
import sys
sys.path.append('../') # or just install the module
sys.path.append('../../flaming-choripan') # or just install the module
sys.path.append('../../astro-lightcurves-handler') # or just install the module

In [2]:
%load_ext autoreload
%autoreload 2
from synthsne import curve_statistics as cs

method = 'mcmc'
rootdir = '../save/alerceZTFv7.1'
cs.get_all_incorrects_fittings(rootdir, method)

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


[]

In [3]:
%load_ext autoreload
%autoreload 2
from synthsne import curve_statistics as cs

method = 'mcmc'
rank, rank_bdict, band_names = cs.get_ranks(rootdir, method)
print(rank)
for b in band_names:
    print(rank_bdict[b])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
mb-rank[top10]:
(1) - ZTF18aasdted: 2542.0933012133173
(2) - ZTF18acmvcme: 1659.653593827637
(3) - ZTF19abclykm: 1573.4311162509025
(4) - ZTF18acyarki: 935.4741315866717
(5) - ZTF20aaurzce: 847.3206695428413
(6) - ZTF19adcecwu: 799.1488034335537
(7) - ZTF18acvwbos: 509.7601000715314
(8) - ZTF19aaeoqst: 493.13563189952595
(9) - ZTF20abchbds: 491.1949079181372
(10) - ZTF20abchhuw: 443.92806873487086
g-rank[top10]:
(1) - ZTF18aasdted: 4026.858871785492
(2) - ZTF18acmvcme: 3293.667300551204
(3) - ZTF18acyarki: 1844.2607733939103
(4) - ZTF19abclykm: 1022.6852167506433
(5) - ZTF19adcecwu: 841.3753725532438
(6) - ZTF18acvwbos: 774.1182936180287
(7) - ZTF20abchhuw: 755.9410681080392
(8) - ZTF20aautzif: 634.634450192937
(9) - ZTF19aatgwhs: 626.2847463331896
(10) - ZTF19aaeoqst: 510.61149788076796
r-rank[top10]:
(1) - ZTF19abclykm: 2124.177015751162
(2) - ZTF20aaurzce: 1686.3501585071488
(3) - ZTF18aasdted: 1

In [49]:
%load_ext autoreload
%autoreload 2
from synthsne import curve_statistics as cs

methods = ['curvefit', 'mcmc', 'uniformprior']
methods = ['curvefit', 'mcmc']
info_df = cs.get_info_dict(rootdir, methods)
info_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,mb-fit-error,mb-ok-fits-n,mb-n,mb-ok-fits[%],g-fit-error,g-ok-fits[%],r-fit-error,r-ok-fits[%]
method-curvefit,83.70±414.17,203544,208896,97.437960,61.93±231.92,96.131089,104.89±534.09,98.744830
method-mcmc,66.66±318.49,203908,208896,97.612209,60.18±359.16,96.425973,72.99±272.88,98.798445


In [38]:
%load_ext autoreload
%autoreload 2
from flamingchoripan.latex.latex_tables import LatexTable

latex_table = LatexTable(info_df)
print(latex_table)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\def\srule{\rule{0pt}{2ex}\rule[-1ex]{0pt}{0ex}}
\begin{table*}
\centering
\caption{???}
\label{???}\vspace{.1cm}
\begin{tabular}{l|cccccc}
\hline
method & mb-fit-error & mb-ok-fits[\%] & g-fit-error & g-ok-fits[\%] & r-fit-error & r-ok-fits[\%] \srule\\\hline\hline
curvefit & 83.70$\pm$414.17 & 97.44 & 61.93$\pm$231.92 & 96.13 & 104.89$\pm$534.09 & 98.74 \srule\\
mcmc & 52.10$\pm$275.46 & 97.80 & 52.49$\pm$371.61 & 97.01 & 51.73$\pm$121.11 & 98.60 \srule\\
\hline
\end{tabular}
\end{table*}
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

